In [4]:
from trulens_eval.tests.test_cases import generate_summeval_groundedness_golden_set


# generator for groundedness golden set
test_cases_gen = generate_summeval_groundedness_golden_set("../../trulens_eval/tests/datasets/summeval/summeval_test_100.json")

In [5]:
test_split, train_split, val_split = [], [] ,[]


for i in range(500):
    test_split.append(next(test_cases_gen))
    
for i in range(500, 1300):
    train_split.append(next(test_cases_gen))

for i in range(1300, 1600):
    val_split.append(next(test_cases_gen))

In [6]:
train_split

[{'query': "Esteban Cambiasso has won all the major European competitions a player can during his illustrious career but revealed that keeping Leicester City in the Premier League would be up there with the best. The Foxes are currently seven points adrift at the bottom of the table, with only eight games remaining, knowing that time is running out to save themselves. Cambiasso refuses to give up and admits that keeping Leicester up will feel like winning a trophy. Esteban Cambiasso says that helping keep Leicester in the Premier League will feel like winning a trophy 'For me, it's like another cup,' he told BBC East Midlands Today. 'When you start another season you have an objective, and this is the objective for us. 'For me, winning a cup or winning the league with another team is the same now as having the possibility to save Leicester in the Premier League.' The Argentinian midfielder poses with the trophy after his team won the 2010 FIFA Club World Cup Cambiasso had an illustriou

In [22]:
import os
os.environ["SNOWFLAKE_ACCOUNT"] = "sfengineering-mlplatformtest"
os.environ["SNOWFLAKE_USER"] = "dhuang"
os.environ["SNOWFLAKE_USER_PASSWORD"] = "Ilovegt@47912301990@"


In [23]:
from trulens_eval.feedback.provider import Cortex
cortex_provider_mixtral = Cortex(model_engine="mixtral-8x7b")


Singleton instance of type CortexEndpoint already created at:
/var/folders/wv/24wjftnn0jb2gk0_90cnrmmh0000gn/T/ipykernel_34424/4034524819.py:2
	cortex_provider_mixtral = Cortex(model_engine="mixtral-8x7b")

You can delete the singleton by calling `<instance>.delete_singleton()` or 
  ```python
  from trulens_eval.utils.python import SingletonPerName
  SingletonPerName.delete_singleton_by_name(name="cortex", cls=CortexEndpoint)
  ```
            


In [24]:
from trulens_eval import Feedback

f_groundedness = Feedback(cortex_provider_mixtral.groundedness_measure_with_cot_reasons, name='groundedness feedback function')

### Extract current prompts we use for groundedness with COT and modify to fit for SFT
1. 

In [33]:
from trulens_eval.feedback import prompts

source = "I love all kinds of food"
hypothesis = "I love fish"
system_prompt: str = prompts.LLM_GROUNDEDNESS_SYSTEM
user_prompt: str =  """SOURCE: {premise}
        
        Hypothesis: {hypothesis}
        
        Please answer with the template below for all statement sentences:

        Criteria: <Statement Sentence>, 
        Score: <Output a number between 0-10 where 0 is no information overlap and 10 is all information is overlapping>
        """.format(
                premise=f"{source}", hypothesis=f"{hypothesis}"
            )

llm_messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]

llm_full_prompt = str(llm_messages)

In [34]:
llm_full_prompt

"[{'role': 'system', 'content': 'You are a INFORMATION OVERLAP classifier; providing the overlap of information between the source and statement.\\n        Respond only as a number from 0 to 10 where 0 is no information overlap and 10 is all information is overlapping.\\n        Never elaborate.'}, {'role': 'user', 'content': 'SOURCE: I love all kinds of food\\n        \\n        Hypothesis: I love fish\\n        \\n        Please answer with the template below for all statement sentences:\\n\\n        Criteria: <Statement Sentence>, \\n        Score: <Output a number between 0-10 where 0 is no information overlap and 10 is all information is overlapping>\\n        '}]"

In [35]:
cortex_provider_mixtral.generate_score_and_reasons(llm_full_prompt,'')

(0.2, {})